# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

# Оглавление 

# Введение
   ## Описание проекта
   ## Цели и задачи
   
# Подготовка данных
   ## Импорт библиотек
   ## Чтение данных
   ## Проверка на пропуски
   
# Разбиение на выборки
   ## Анализ данных
   
# Обучение моделей
   ## Дерево решений
   ## Случайный лес
   ## Логистическая регрессия
   
# Оценка моделей
   ## Проверка моделей на тестовой выборке
   
# Проверка моделей на адекватность

# Общие выводы

Целью проекта является построение модели классификации для определения подходящего тарифа на основе данных о поведении клиентов оператора связи.

Для достижения этой цели были поставлены следующие задачи:

Загрузка и предобработка данных;
Анализ данных и изучение основных закономерностей поведения клиентов;
Подготовка признаков для обучения модели;
Обучение нескольких моделей машинного обучения и выбор лучшей;
Оценка качества выбранной модели на тестовой выборке;
Проверка выбранной модели на адекватность.
Для достижения целей проекта использовались библиотеки Python для анализа данных и машинного обучения, такие как Pandas, NumPy и Scikit-learn.

# Подготовка данных

In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


Данный датасет содержит информацию о поведении пользователей мобильного оператора за один месяц. Каждая строка - это данные об одном пользователе, включая количество звонков (calls), суммарную длительность звонков в минутах (minutes), количество sms-сообщений (messages), объем израсходованного интернет-трафика в мегабайтах (mb_used) и тариф, которым пользовался пользователь в течение месяца (is_ultra). Если значение is_ultra равно 1, то пользователь пользовался тарифом "Ультра", если значение равно 0, то тарифом "Смарт".

In [3]:
df.shape

(3214, 5)

Это значит, что в датасете содержится информация о 3214 клиентах, которые использовали мобильную связь в течение месяца.

In [4]:
# Проверка на пропуски
# Общее количество пропусков по столбцам
df.isnull().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

Это значит, что данные полные и готовы для анализа. 

# Разбиение на выборки

In [5]:
# Разбиваем данные на обучающую, валидационную и тестовую выборки в соотношении 3:1:1
df_train_valid, df_test = train_test_split(df, test_size=0.2, random_state=12345)
df_train, df_valid = train_test_split(df_train_valid, test_size=0.25, random_state=12345)


# Выводим размеры получившихся выборок
print("Размер обучающей выборки:", len(df_train))
print("Размер валидационной выборки:", len(df_valid))
print("Размер тестовой выборки:", len(df_test))

Размер обучающей выборки: 1928
Размер валидационной выборки: 643
Размер тестовой выборки: 643


Разбиение данных на обучающую, тестовую и валидационную выборки является важной частью процесса машинного обучения. Это позволяет оценить качество модели на новых данных и избежать переобучения. В данном проекте данные были разбиты на обучающую (60%), тестовую (20%) и валидационную (20%) выборки с помощью метода train_test_split из библиотеки scikit-learn.

# Обучение моделей

Для начала необходимо выделить признаки и целевой признак в обучающей выборке, валидационной выборке и тестовой выборке.

Выделим целевой признак is_ultra, а остальные признаки запишем в переменную features.

In [6]:
features_train = df_train.drop('is_ultra', axis=1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop('is_ultra', axis=1)
target_valid = df_valid['is_ultra']
features_test = df_test.drop('is_ultra', axis=1)
target_test = df_test['is_ultra']

Затем создадим 3 модели и обучим их на обучающей выборке: дерево решений, случайный лес и логистическую регрессию.

In [7]:
# Дерево решений
from sklearn.tree import DecisionTreeClassifier
tree_model = DecisionTreeClassifier(random_state=12345)
tree_model.fit(features_train, target_train)

DecisionTreeClassifier(random_state=12345)

In [8]:
# Случайный лес
from sklearn.ensemble import RandomForestClassifier
forest_model = RandomForestClassifier(random_state=12345, n_estimators=100, max_depth = 5)
forest_model.fit(features_train, target_train)

RandomForestClassifier(max_depth=5, random_state=12345)

In [9]:
# Логистическая регрессия
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression(random_state=12345, solver='liblinear')
lr_model.fit(features_train, target_train)

LogisticRegression(random_state=12345, solver='liblinear')

Для модели случайного леса можно подобрать следующие гиперпараметры:

n_estimators - количество деревьев в лесу

max_depth - максимальная глубина дерева

min_samples_leaf - минимальное количество объектов, необходимых для создания листа

Можно воспользоваться методом GridSearchCV из библиотеки sklearn, чтобы перебрать все комбинации гиперпараметров и выбрать наилучшие. Например, таким образом можно подобрать три гиперпараметра: n_estimators, max_depth и min_samples_leaf.

Вот пример кода для подбора гиперпараметров с помощью GridSearchCV:

In [10]:
from sklearn.model_selection import GridSearchCV

# зададим список значений гиперпараметров для перебора
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15],
    'min_samples_leaf': [1, 3, 5]
}

# создадим модель случайного леса
forest_model = RandomForestClassifier(random_state=12345)

# создадим объект GridSearchCV
grid_search = GridSearchCV(forest_model, param_grid, cv=5)

# запустим поиск лучших гиперпараметров
grid_search.fit(features_train, target_train)

# выведем лучшие гиперпараметры и значение метрики accuracy
print('Лучшие гиперпараметры:', grid_search.best_params_)
print('Accuracy на кросс-валидации:', grid_search.best_score_)

Лучшие гиперпараметры: {'max_depth': 10, 'min_samples_leaf': 1, 'n_estimators': 200}
Accuracy на кросс-валидации: 0.8189852634412219


Этот код переберает все комбинации заданных значений гиперпараметров, обучает модель на каждой комбинации и оценивает качество с помощью кросс-валидации. После завершения работы GridSearchCV выводит лучшие гиперпараметры и соответствующее значение метрики accuracy на кросс-валидации.

Для решающего дерева можно подобрать следующие гиперпараметры:

max_depth: максимальная глубина дерева

min_samples_split: минимальное число образцов, необходимых для разделения внутреннего узла

min_samples_leaf: минимальное число образцов, необходимых для быть листовым узлом

Ниже представлен код для подбора гиперпараметров с помощью GridSearchCV:

In [11]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# зададим список значений гиперпараметров для перебора
param_grid = {
    'max_depth': [2, 4, 6, 8],
    'min_samples_split': [2, 4, 6, 8],
    'min_samples_leaf': [1, 3, 5, 7]
}

# создадим модель дерева решений
tree_model = DecisionTreeClassifier(random_state=12345)

# создадим объект GridSearchCV
grid_search = GridSearchCV(tree_model, param_grid, cv=5)

# запустим поиск лучших гиперпараметров
grid_search.fit(features_train, target_train)

# выведем лучшие гиперпараметры и значение метрики accuracy
print('Лучшие гиперпараметры:', grid_search.best_params_)
print('Accuracy на кросс-валидации:', grid_search.best_score_)

Лучшие гиперпараметры: {'max_depth': 4, 'min_samples_leaf': 7, 'min_samples_split': 2}
Accuracy на кросс-валидации: 0.8034237265325348


# Оценка моделей

Модель случайного леса:

In [12]:
forest_pred = grid_search.best_estimator_.predict(features_test)
forest_accuracy = accuracy_score(target_test, forest_pred)
print('Accuracy на тестовой выборке:', forest_accuracy)

Accuracy на тестовой выборке: 0.7916018662519441


Модель дерева решений: 

In [13]:
# импортируем нужные модули
from sklearn.metrics import accuracy_score

# получим предсказания модели на тестовых данных
tree_pred = grid_search.predict(features_test)

# вычислим accuracy на тестовых данных
tree_acc = accuracy_score(target_test, tree_pred)

# выведем значение accuracy
print('Accuracy на тестовой выборке:', tree_acc)

Accuracy на тестовой выборке: 0.7916018662519441


Модель логистической регрессии: 

In [14]:
logreg_predictions = lr_model.predict(features_test)
logreg_accuracy = accuracy_score(target_test, logreg_predictions)
print("Accuracy на тестовой выборке:", logreg_accuracy)

Точность модели логистическая регрессия на тестовой выборке: 0.6998444790046656


Модель случайного леса и модуль решающего дерева показали наилучшее качество на тестовой выборке с долей правильных ответов около 80%, что является хорошим результатом. Логистическая регрессия в данной задаче показала наихудший результат.

    
<div class="alert-warning"> 
<b>Комментарий ревьюера 💡 </b> 
    
accuracy - это не точность, а доля правильных ответов. Точность - другая метрика классификации, precision

# Проверка моделей на адекватность

Для проверки моделей на адекватность мы можем сравнить их метрики с метриками константных моделей. Константная модель всегда предсказывает одно и то же значение (например, среднее значение целевого признака в тренировочной выборке).

Для нашего случая, у нас два класса: 0 и 1, и они примерно сбалансированы. Таким образом, мы можем использовать константную модель, которая будет всегда предсказывать наиболее часто встречающийся класс, т.е. 0.

Для оценки метрик константной модели, мы можем использовать функцию accuracy_score из библиотеки scikit-learn.
Давайте посмотрим на результаты:

In [15]:
# константная модель
constant_predictions = [0] * len(target_test)

# accuracy константной модели
constant_accuracy = accuracy_score(target_test, constant_predictions)

print("Accuracy константной модели: {:.2f}".format(constant_accuracy))

Accuracy константной модели: 0.70


Теперь мы можем сравнить метрики наших моделей с метриками константной модели. Если метрики наших моделей не значительно лучше метрик константной модели, то мы не можем утверждать, что наши модели адекватны.

В нашем случае, лучшая метрика accuracy равна 0.80 (у модели случайного леса и модели решающего дерева), что значительно превосходит метрику константной модели (0.70). Поэтому мы можем утверждать, что наши модели адекватны.

# Общие выводы
В ходе проекта была решена задача классификации клиентов мобильного оператора на два тарифа: "Смарт" и "Ультра". Для этого был проведен анализ данных, заполнены пропущенные значения и произведен их предобработка. Затем данные были разбиты на обучающую, валидационную и тестовую выборки.

Для решения задачи были построены и исследованы три модели: дерево решений, случайный лес и логистическая регрессия. Наилучший результат был достигнут с помощью модели случайного леса и дерева решений со значением метрики accuracy на тестовой выборке в 79,1 %, что превышает необходимый уровень точности в 75%.

Также была произведена проверка полученной модели на адекватность, и она показала, что модель работает лучше, чем случайное угадывание.

Таким образом, мы можем сделать вывод, что модель случайного леса и модель решающего дерева является наилучшей для решения данной задачи, и ее можно использовать для классификации новых клиентов на тарифы "Смарт" и "Ультра".

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [ ] Весь код исполняется без ошибок
- [ ] Ячейки с кодом расположены в порядке исполнения
- [ ] Выполнено задание 1: данные загружены и изучены
- [ ] Выполнено задание 2: данные разбиты на три выборки
- [ ] Выполнено задание 3: проведено исследование моделей
    - [ ] Рассмотрено больше одной модели
    - [ ] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [ ] Написаны выводы по результатам исследования
- [ ] Выполнено задание 3: Проведено тестирование
- [ ] Удалось достичь accuracy не меньше 0.75
